Cargo los datos sobre temperaturas durante el año 2024 de Barcelona desde la estación de Drassanes que tiene un criterio de temperatura a nivel ciudad.

In [1]:
import pandas as pd

# Cargar los archivos CSV
df1 = pd.read_csv('temp_012024_062024.csv')
df2 = pd.read_csv('temp_072024_122024.csv')

# Concatenar los DataFrames por filas
df_temp_2024 = pd.concat([df1, df2], axis=0, ignore_index=True)

# Mostrar el DataFrame combinado
df_temp_2024.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2024-01-01,0201X,"BARCELONA, DRASSANES",BARCELONA,11,"11,8","0,0","8,8",23:52,"14,7",14:45,68.0,76.0,05:00,61.0,10:50
1,2024-01-02,0201X,"BARCELONA, DRASSANES",BARCELONA,11,"11,6","0,0","7,9",03:53,"15,3",15:59,46.0,66.0,00:00,27.0,16:00
2,2024-01-03,0201X,"BARCELONA, DRASSANES",BARCELONA,11,"15,4","0,0","12,6",07:23,"18,2",12:22,61.0,71.0,07:40,50.0,01:30
3,2024-01-04,0201X,"BARCELONA, DRASSANES",BARCELONA,11,"14,2","0,4","11,6",07:02,"16,9",13:18,79.0,85.0,08:40,65.0,00:00
4,2024-01-05,0201X,"BARCELONA, DRASSANES",BARCELONA,11,"11,6","8,8","9,6",Varias,"13,6",12:46,79.0,89.0,04:30,67.0,13:30


El dataframe está compuesto por 366 filas y 16 columnas

In [2]:
df_temp_2024.shape 

(366, 16)

Hay columnas que no contemplo utilizar por lo que decido eliminarlas para aligerar el dataframe.

In [3]:
df_temp_2024 = df_temp_2024.drop(columns=['hrMedia', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin'])

Reviso las características de cada columna, observo varios detalles sobre ellas que considero tratar:

* `fecha` es del tipo `object` y el formato debería ser `datetime`
* Hay varias columas que aparecen como tipo `object` al utiizar una coma como decimal y las transformaré como float con punto.


In [4]:
df_temp_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   fecha       366 non-null    object
 1   indicativo  366 non-null    object
 2   nombre      366 non-null    object
 3   provincia   366 non-null    object
 4   altitud     366 non-null    int64 
 5   tmed        365 non-null    object
 6   prec        366 non-null    object
 7   tmin        365 non-null    object
 8   horatmin    365 non-null    object
 9   tmax        365 non-null    object
 10  horatmax    365 non-null    object
dtypes: int64(1), object(10)
memory usage: 31.6+ KB


Realizo la transformación de la columna `fecha` al tipo `datetime`

In [5]:
df_temp_2024['fecha']= pd.to_datetime(df_temp_2024['fecha'], format='%Y-%m-%d')
print(df_temp_2024.dtypes)

fecha         datetime64[ns]
indicativo            object
nombre                object
provincia             object
altitud                int64
tmed                  object
prec                  object
tmin                  object
horatmin              object
tmax                  object
horatmax              object
dtype: object


Transformo las columnas 'tmed', 'prec', 'tmin', 'tmax' al tipo float, modificando el símbolo de coma por punto.

In [6]:

columnas_a_float = ['tmed', 'prec', 'tmin', 'tmax']

# Reemplazo las comas por puntos para que pueda realizarse el cambio al tipo float
for col in columnas_a_float:
    df_temp_2024[col] = df_temp_2024[col].str.replace(',', '.')  

df_temp_2024[columnas_a_float] = df_temp_2024[columnas_a_float].astype(float)

print(df_temp_2024.dtypes)

fecha         datetime64[ns]
indicativo            object
nombre                object
provincia             object
altitud                int64
tmed                 float64
prec                 float64
tmin                 float64
horatmin              object
tmax                 float64
horatmax              object
dtype: object


In [7]:
df_temp_2024.isnull().sum()

fecha         0
indicativo    0
nombre        0
provincia     0
altitud       0
tmed          1
prec          0
tmin          1
horatmin      1
tmax          1
horatmax      1
dtype: int64

Analizo el valor nulo de la variable `tmed`, observo que aparecen varias columnas en ese registro(337) con valores faltantes en las columnas mostradas en la revisión de los valores faltantes en el dataframe.

In [8]:
nulos_tmed = df_temp_2024['tmed'].isnull()
df_temp_2024[nulos_tmed]

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax
337,2024-12-03,0201X,"BARCELONA, DRASSANES",BARCELONA,11,NaN,0.6,NaN,NaN,NaN,NaN


La idea de la imputación en este caso es mantener la continuidad de la serie. Imputo los valores faltantes utilizando una función, que en caso de que encuentre un valor nulo en una columna, buscará el valor correspondiente en la fila anterior (día anterior). Si el valor anterior existe, se utiliza para reemplazar el valor nulo. 

In [9]:

# Ordeno el DataFrame por fecha para asegurar la correcta imputación
df_temp_2024 = df_temp_2024.sort_values('fecha')

# Selecciono las columnas a imputar 
columnas_a_imputar = ['tmed', 'tmin', 'horatmin', 'tmax', 'horatmax']

# Se itera sobre las filas y las columnas a imputar
for index, row in df_temp_2024.iterrows():
    for columna in columnas_a_imputar:
        if pd.isna(row[columna]):
            # Buscar el valor del día anterior en la misma columna
            valor_anterior = df_temp_2024.loc[index - 1, columna]
            # Imputar si el valor anterior existe
            if not pd.isna(valor_anterior):
                df_temp_2024.loc[index, columna] = valor_anterior

Compruebo que los valores ha sido imputados totalmente.

In [10]:
df_temp_2024.isnull().sum()

fecha         0
indicativo    0
nombre        0
provincia     0
altitud       0
tmed          0
prec          0
tmin          0
horatmin      0
tmax          0
horatmax      0
dtype: int64

He observado también que las columnas donde aparecen las horas no se encuentran en el formato de hora, cuando intento cambiar el tipo de dato me aparece un error al intentarlo. Observo el dataframe y detecto que aparece la palabra `Varias` en algún registro de las horas, por lo que busco en esa columna para ver si existen más.

In [11]:
palabra_a_buscar = "Varias"
df_temp_2024[df_temp_2024['horatmin'].str.contains(palabra_a_buscar, case=False)]

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax
4,2024-01-05,0201X,"BARCELONA, DRASSANES",BARCELONA,11,11.6,8.8,9.6,Varias,13.6,12:46
6,2024-01-07,0201X,"BARCELONA, DRASSANES",BARCELONA,11,11.2,0.0,6.8,Varias,15.5,15:06
46,2024-02-16,0201X,"BARCELONA, DRASSANES",BARCELONA,11,14.4,1.8,13.1,Varias,15.7,10:51
54,2024-02-24,0201X,"BARCELONA, DRASSANES",BARCELONA,11,12.6,0.0,9.2,Varias,16.0,14:28
55,2024-02-25,0201X,"BARCELONA, DRASSANES",BARCELONA,11,12.2,0.0,9.2,Varias,15.3,11:30
60,2024-03-01,0201X,"BARCELONA, DRASSANES",BARCELONA,11,14.8,0.0,10.1,Varias,19.6,15:15
76,2024-03-17,0201X,"BARCELONA, DRASSANES",BARCELONA,11,15.9,0.0,13.6,Varias,18.2,10:44
81,2024-03-22,0201X,"BARCELONA, DRASSANES",BARCELONA,11,19.0,0.0,14.2,Varias,23.8,12:18
105,2024-04-15,0201X,"BARCELONA, DRASSANES",BARCELONA,11,19.2,0.0,15.7,Varias,22.6,12:36
108,2024-04-18,0201X,"BARCELONA, DRASSANES",BARCELONA,11,14.8,0.0,12.6,Varias,17.1,11:21


En el resultado me aparecen varias columnas de horas con la palabra dentro del dataframe (`horatim` y `horatmax`), por lo que decido realizar el método anterior de imputación para resolver esta anomalía en los datos.

In [12]:
# Ordeno el DataFrame por fecha para asegurar la correcta imputación
df_temp_2024 = df_temp_2024.sort_values('fecha')

# Selecciono las columnas a imputar 
columnas_a_imputar = ['horatmin', 'horatmax']

# Iteramos sobre las filas y las columnas a imputar
for index, row in df_temp_2024.iterrows():
    for columna in columnas_a_imputar:
        if row[columna] == "Varias":
            # Buscamos el valor del día anterior (índice anterior)
            valor_anterior = df_temp_2024.loc[index - 1, columna]
            df_temp_2024.loc[index, columna] = valor_anterior

print(df_temp_2024.dtypes)

fecha         datetime64[ns]
indicativo            object
nombre                object
provincia             object
altitud                int64
tmed                 float64
prec                 float64
tmin                 float64
horatmin              object
tmax                 float64
horatmax              object
dtype: object


Ahora realizo el cambio del tipo de datos en las columnas de las horas

In [13]:
columnas_hora = ['horatmin', 'horatmax']

for columna in columnas_hora:
    df_temp_2024[columna] = pd.to_datetime(df_temp_2024[columna], format='%H:%M', errors='coerce')

print(df_temp_2024.dtypes)

fecha         datetime64[ns]
indicativo            object
nombre                object
provincia             object
altitud                int64
tmed                 float64
prec                 float64
tmin                 float64
horatmin      datetime64[ns]
tmax                 float64
horatmax      datetime64[ns]
dtype: object


In [14]:
df_temp_2024.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax
0,2024-01-01,0201X,"BARCELONA, DRASSANES",BARCELONA,11,11.8,0.0,8.8,1900-01-01 23:52:00,14.7,1900-01-01 14:45:00
1,2024-01-02,0201X,"BARCELONA, DRASSANES",BARCELONA,11,11.6,0.0,7.9,1900-01-01 03:53:00,15.3,1900-01-01 15:59:00
2,2024-01-03,0201X,"BARCELONA, DRASSANES",BARCELONA,11,15.4,0.0,12.6,1900-01-01 07:23:00,18.2,1900-01-01 12:22:00
3,2024-01-04,0201X,"BARCELONA, DRASSANES",BARCELONA,11,14.2,0.4,11.6,1900-01-01 07:02:00,16.9,1900-01-01 13:18:00
4,2024-01-05,0201X,"BARCELONA, DRASSANES",BARCELONA,11,11.6,8.8,9.6,1900-01-01 07:02:00,13.6,1900-01-01 12:46:00


- Elimino las columnas `indicativo`, `nombre`, `horatmin`, `horatmax`.
- Renombro las columna `provincia` por `ciudad`.

In [15]:
df_temp_2024= df_temp_2024.drop(columns= ['indicativo', 'nombre', 'horatmin','horatmax'])
df_temp_2024= df_temp_2024.rename(columns={'provincia': 'ciudad'})
df_temp_2024.head()


,fecha,ciudad,altitud,tmed,prec,tmin,tmax
0,2024-01-01,BARCELONA,11,11.8,0.0,8.8,14.7
1,2024-01-02,BARCELONA,11,11.6,0.0,7.9,15.3
2,2024-01-03,BARCELONA,11,15.4,0.0,12.6,18.2
3,2024-01-04,BARCELONA,11,14.2,0.4,11.6,16.9
4,2024-01-05,BARCELONA,11,11.6,8.8,9.6,13.6


Hay elementos que a pesar de modificarlos porque pensaba utilizarlos, al final los elimino, puedo pensar que es un paso que podía haberme ahorrado, pero el proceso de modificación/imputación que les apliqué me resultan importantes mantenerlos porque puedo utilizarlos en el desarrollo de este proyecto.

Importo el dataframe a un archivo csv

In [16]:
df_temp_2024.to_csv('temp_bcn_2024.csv', index= False)